In [49]:
import os
from PIL import Image, ImageFilter, ImageOps
import numpy as np
import random

In [50]:
def get_image_size(image_path):
    with Image.open(image_path) as img:
        return img.size

In [51]:
def check_image_sizes(directory, output_file):
    image_sizes = {}
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
                file_path = os.path.join(root, file)
                try:
                    size = get_image_size(file_path)
                    if size in image_sizes:
                        image_sizes[size].append(file_path)
                    else:
                        image_sizes[size] = [file_path]
                except Exception as e:
                    print(f"Could not process image {file_path}: {e}", file=output_file)

    if len(image_sizes) == 1:
        print("All images are the same size.", file=output_file)
    else:
        print("Images have different sizes.", file=output_file)
        for size, paths in image_sizes.items():
            print(f"Size {size}: {len(paths)} images", file=output_file)

In [52]:
# Info text file on the different image sizes in the raw dataset
directory = 'dataset5/'
output_file_path = 'imagesizes.txt'

with open(output_file_path, 'w') as output_file:
    check_image_sizes(directory, output_file)

print(f"Results written to {output_file_path}")

Results written to imagesizes.txt


In [53]:
def apply_custom_sharpen(image):
    kernel = [
        0, -1, 0,
        -1, 6, -1,
        0, -1, 0
    ]
    # Apply the kernel to the image
    sharpened_image = image.filter(ImageFilter.Kernel((3, 3), kernel, scale=1, offset=0))
    return sharpened_image

In [54]:
def add_salt_and_pepper_noise(image, amount=0.05, salt_vs_pepper=0.5):
    image_array = np.array(image)
    row, col = image_array.shape
    num_salt = np.ceil(amount * image_array.size * salt_vs_pepper)
    num_pepper = np.ceil(amount * image_array.size * (1.0 - salt_vs_pepper))

    # salt
    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image_array.shape]
    image_array[coords[0], coords[1]] = 1

    # pepper
    coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image_array.shape]
    image_array[coords[0], coords[1]] = 0

    return Image.fromarray(image_array)

In [55]:
def random_rotation(image, angle_range=(-10, 10)):
    angle = random.uniform(angle_range[0], angle_range[1])
    return image.rotate(angle)

In [56]:
def random_zoom(image, zoom_range=(0.9, 1.1)):
    zoom_factor = random.uniform(zoom_range[0], zoom_range[1])
    width, height = image.size
    new_width, new_height = int(width * zoom_factor), int(height * zoom_factor)
    image = image.resize((new_width, new_height), Image.LANCZOS)
    if zoom_factor < 1:
        pad_width = (width - new_width) // 2
        pad_height = (height - new_height) // 2
        image = ImageOps.expand(image, (pad_width, pad_height, width - new_width - pad_width, height - new_height - pad_height), fill=0)
    else:
        left = (new_width - width) // 2
        top = (new_height - height) // 2
        image = image.crop((left, top, left + width, top + height))
    return image

In [57]:
def random_shift(image, width_shift_range=0.1, height_shift_range=0.1):
    width, height = image.size
    max_width_shift = int(width * width_shift_range)
    max_height_shift = int(height * height_shift_range)
    width_shift = random.randint(-max_width_shift, max_width_shift)
    height_shift = random.randint(-max_height_shift, max_height_shift)
    return image.transform(image.size, Image.AFFINE, (1, 0, width_shift, 0, 1, height_shift), resample=Image.BICUBIC)


In [58]:
def random_horizontal_flip(image):
    if random.random() < 0.5:
        image = ImageOps.mirror(image)
    return image

In [59]:
def random_crop(image, crop_size=(50, 50)):
    width, height = image.size
    new_width, new_height = crop_size
    left = random.randint(0, width - new_width)
    top = random.randint(0, height - new_height)
    return image.crop((left, top, left + new_width, top + new_height))

In [60]:
def preprocess_image(image_path):
    image = Image.open(image_path).convert('L')
    
    image = random_rotation(image)
    image = random_zoom(image)
    image = random_shift(image)
    image = random_horizontal_flip(image)
    
    resized_image = image.resize((50, 50))
    
    sharpened_image = apply_custom_sharpen(resized_image)
    
    noisy_image = add_salt_and_pepper_noise(sharpened_image)
    
    normalized_image = np.array(noisy_image) / 255.0
    
    return Image.fromarray((normalized_image * 255).astype(np.uint8))

In [61]:
def process_directory(input_dir, output_dir):
    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.lower().startswith('color') and file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
                file_path = os.path.join(root, file)
                
                relative_path = os.path.relpath(root, input_dir)
                output_path = os.path.join(output_dir, relative_path, file)
                
                os.makedirs(os.path.dirname(output_path), exist_ok=True)
                
                # Preprocess and save 
                try:
                    processed_image = preprocess_image(file_path)
                    processed_image.save(output_path)
                except Exception as e:
                    print(f"Could not process image {file_path}: {e}")

In [62]:
input_directory = 'dataset5'
output_directory = 'dataset5_preprocessed_50x50'
process_directory(input_directory, output_directory)
print("hehehe finished")

hehehe finished
